In [1]:

using POMDPs, QuickPOMDPs, MCTS, DiscreteValueIteration, POMDPSimulators, POMDPModels, POMDPPolicies, POMDPModelTools
using Distributions, Combinatorics, StaticArrays, Statistics
using FileIO, JLD2, TickTock
global Number_level = 10;

## State Functions

In [2]:
function state_cnt(n, S) #n = number of units; S = number of states
    if S==1
        return 1
    end
    return binomial(n+S-1, S-1)
end

function state_index(S,s) #s = state vector
    #S = length(s)
    if S == 1
        return 1
    end
    if s[S]==0
        return state_index(S-1, s[1:(S-1)])
    end
    
    n_prev = sum(s)
    prev = state_cnt(n_prev, S-1) # count of all states with s[S]=0
    inc = prev
    for i in 1:(s[S]-1)
        inc = inc/(n_prev+S-2)*n_prev #count of all states with s[S]=i
        prev = prev + inc
        n_prev = n_prev - 1
    end
    return prev + state_index(S-1, s[1:(S-1)])
end

function state_vec(n, S, ind) # n = number of units; S = number of states; ind = state index
    if ind < 0.5 || ind > state_cnt(n, S) + 0.5
        println("index is out of range!")
        return -1
    end
    if S==1 
        return [n]
    end
    if ind < state_cnt(n, S-1) + 0.5
        return push!(state_vec(n, S-1, ind), 0)
    end
    prev = state_cnt(n, S-1)
    inc = prev
    last_state = 0
    n_prev = n
    while ind > prev + 0.5
        inc = inc/(n_prev+S-2)*n_prev #count of all sta_tes with s[S]=last_state
        prev += inc
        n_prev -= 1
        last_state += 1
    end
    return push!(state_vec(n-last_state, S-1, ind - prev + inc), last_state)
end

#using hueristic rule for rollout
mutable struct nNRollout <: Policy
    n::Int64
    N::Int64
end

mutable struct nmNRollout <: Policy
    n::Int64
    m::Int64
    N::Int64
end

mutable struct mystate
    state::Vector{Int64};
end

In [3]:
function POMDPs.action(p::nNRollout, s::mystate)
        local_a = 0;
        local_s = s.state;
        nN = sample(1:Number_level, 2, replace = true);
        p.N = maximum(nN);
        p.n = minimum(nN);
        if sum(local_s[p.N:Number_level])>=1
        local_a = p.n;
        end
        return local_a; 
end

function POMDPs.action(p::nmNRollout, s::mystate)
    local_a = 0;
    local_s = s.state;
    nmN = sample(1:Number_level, 3, replace = true);
    p.N = maximum(nmN);
    p.n = minimum(nmN);
    p.m = sum(nmN)-p.N-p.n;
    if sum(local_s[p.N:Number_level])>=1 || sum(local_s[p.m:Number_level])>=2
        local_a = p.n;
    end
    return local_a; 
end

In [4]:
function findNn()
    println("Finding best nN policy...")
for N in 2:(Number_level)
    for n in 1:N
       trials=10000;
        simsteps = 100;
        results=zeros(trials)*0.1
        Threads.@threads for h in 1:trials
        s=repeat(1:1,NumberUnits)
        r1=[]
        a1=[]
        nN=[n,N]
        r=repeat(1:1,NumberUnits)*1.0
        failedcnt = 0 # count failed components
         for k in 1:simsteps
            a =decison(nN,s)
                    r2=0.0
            if 1 in a # if any replacement occurs
            for i in 1:NumberUnits
                if s[i] == Number_level
                    failedcnt += 1
                end
                 s[i],r[i]=generative(s[i],a[i]+2,i,234); 
                 # a=2 in generative means component continueing; a=3 in generative means component replaced
                  r2+=r[i]
                    end 
            else # if no replacement 
               for i in 1:NumberUnits
                if s[i] == Number_level
                    failedcnt += 1
                end
                 s[i],r[i]=generative(s[i],1,i,234);

                  r2+=r[i]
                    end              
            end
            if failedcnt >= limit # system fails
                r2 += system_penalty
            end
            s=s;
            append!(a1,a)
            append!(r1,r2)
        end
        rk=0
        for i in 1:simsteps
            rk=0.95*rk+r1[simsteps-i+1]
        end
            results[h]=rk
        end
        rewards_nN[n,N] = mean(results);
            rewards_nN_std[n,N] = var(results)^(0.5);
    end
end
(max_rward,nN) = findmax(rewards_nN);
println("Max rewards of (N, n) policy is ", max_rward, "  n is ", nN[1], "  N is ",nN[2]);
return  max_rward,rewards_nN_std[nN[1],nN[2]],nN;
end


findNn (generic function with 1 method)

In [5]:
# decisions are made given (n, N)
function decison(nN, s,n=0.2)
    a_final=[]
     if length(s[s .>= nN[2]])>0
         for i in 1:NumberUnits
             append!(a_final,Int(s[i]>=nN[1]))
             end
         return a_final
         # a_final is binary vector
     end
     a_0=repeat(0:0,NumberUnits);
      return a_0
 end

decison (generic function with 2 methods)

In [6]:
Number_level=10
T=zeros(Number_level,Number_level,3,100);

for i in 1:100
    #fullname = "/Users/huhan/Library/CloudStorage/GoogleDrive-hhu98@wisc.edu/My Drive/Master's Thesis/Heterogeneous/TM_MATRIX/tm"*string(i);
    fullname = "J:/Master's Thesis/Heterogeneous/TM_MATRIX/tm"*string(i);
    # generate_transition_matrix(Number_level, fullname) 
    Transition_matrix1 = load(fullname*".jld2","transition_matrix");
    
    T[:,:,1,i].=Transition_matrix1;
    T[:,:,2,i].=Transition_matrix1;
    for j in 1:10
        T[j,:,3,i].=Transition_matrix1[1,:];
        end
    T[10,:,1,i].=Transition_matrix1[1,:]
T[10,:,2,i].=Transition_matrix1[1,:]
end

In [7]:
T[:,:,1,1]

10×10 Matrix{Float64}:
 0.232649  0.229122  0.130585  0.0954644  …  0.0507595  0.0416229  0.00861626
 0.0       0.218724  0.163234  0.128353      0.0899794  0.0431818  0.00868773
 0.0       0.0       0.166395  0.182476      0.111027   0.0773355  0.043199
 0.0       0.0       0.0       0.269557      0.125296   0.0981466  0.0533485
 0.0       0.0       0.0       0.0           0.125828   0.109077   0.0611228
 0.0       0.0       0.0       0.0        …  0.215491   0.17866    0.0619123
 0.0       0.0       0.0       0.0           0.352355   0.221208   0.0646198
 0.0       0.0       0.0       0.0           0.471891   0.449917   0.0781922
 0.0       0.0       0.0       0.0           0.0        0.591116   0.408884
 0.232649  0.229122  0.130585  0.0954644     0.0507595  0.0416229  0.00861626

In [8]:
function generative(s, a,k, rng=23)       #s is a vector of number units at each level and a is the number of units we will repair
    #     if s==Number_level # component fails
    #     s=1
    #     crd = Categorical(T[s,:,1,k]);
    #         s = rand(crd);
    #         r=failure_penalty + normal_operation +  setup_cost/NumberUnits;
    #     return (sp=s, r=r)
    #    end
        r=0
     if a==1 # no replacement in system
            crd = Categorical(T[s,:,1,k]);
            s = rand(crd);
            r = normal_operation ;
         return (sp=s, r=r)
        end
    if a==2 # replacement occurs in system, component not replaced
            crd = Categorical(T[s,:,1,k]); # transition same as a=1
            s = rand(crd);
            r = normal_operation +  setup_cost/NumberUnits ;
             return (sp=s, r=r)
        end
    if a==3 # replacement occurs in system, component replaced 
      
        if s==Number_level
        s=1
        crd = Categorical(T[s,:,3,k]);
        s = rand(crd);
       
        r=failure_penalty + normal_operation +  setup_cost/NumberUnits ;
        return (sp=s, r=r)
        else
        s1=deepcopy(s)
        s=1
        r = maintenance_penalty+  setup_cost/NumberUnits
        crd = Categorical(T[s,:,3,k]);
        s = rand(crd);
        r+= normal_operation  ;
#            r*=(s1/10);
        return (sp=s, r=r)
#         end
end
    end
end

generative (generic function with 2 methods)

In [9]:
using DataFrames
df= DataFrame(u=[],K=[],n=[],s=[],m=[],f=[],p=[],lamb=[],mean=[],std=[])

cost=[ [0	-200	-100	-1200 -2000],
    [0	-400	-100	-1200 -2000],
    [0	-600	-100	-1200 -2000],
    [0	-800	-100	-1200 -2000],
    [0	-1000	-100	-1200 -2000],
    [0	-1200	-100	-1200 -2000],
    [0	-1400	-100	-1200 -2000],
    [0	-1600	-100	-1200 -2000],
    [0	-1800	-100	-1200 -2000],
    [0	-2000	-100	-1200 -2000],
    [0  -2400   -100    -1200 -2000],
    [0  -2800   -100    -1200 -2000],
    [0  -3200   -100    -1200 -2000]
] 


for units in [60,50,40,30,25,20]
    for K in 2:2:convert(Int64, floor((units+1)/2))
 
    global NumberUnits=convert(Int64,units)


# global fullname = "tm10.jld2";
# fullname = "tm10";
global limit=convert(Int64,K)

for c in cost
    tick()
   print(units)  
   print("K : ")
   println(K)
   global normal_operation,setup_cost,maintenance_penalty,failure_penalty,system_penalty=c
   println(c)   
   global  simsteps = 100;
global repetition = 10000;
global rewards_nN = zeros(Number_level,Number_level);
global rewards_nN = rewards_nN.+(-100000000.0);
global rewards_nN_std = zeros(Number_level,Number_level);
global rewards_nN_std = rewards_nN_std.+(-100000000.0);
global discount_factor =0.95
global temp_rewards = zeros(repetition,1);

rewards,std, nN=findNn()

trials=10000;
simsteps = 100;
results=zeros(trials)*0.1
Threads.@threads for h in 1:trials
s=repeat(1:1,NumberUnits)
r1=[]
a1=[]   
r=repeat(1:1,NumberUnits)*1.0
 for k in 1:simsteps
    a =decison(nN,s)
            r2=0.0
            failedcnt = 0 
    if 1 in a
    for i in 1:NumberUnits
        if s[i] == Number_level
            failedcnt += 1
        end
         s[i],r[i]=generative(s[i],a[i]+2,i,234);
          r2+=r[i]
            end 
    else
       for i in 1:NumberUnits
        if s[i] == Number_level
            failedcnt += 1
        end
         s[i],r[i]=generative(s[i],1,i,234);
          r2+=r[i]
            end              
    end
    s=s;
    if failedcnt >= limit # system fails
        r2 += system_penalty
    end
    append!(a1,a)
    append!(r1,r2)
end
rk=0
for i in 1:simsteps
    rk=0.95*rk+r1[simsteps-i+1]
end
    results[h]=rk
end
println(units)
        println(var(results)^0.5)
println(mean(results))


pushfirst!(df,[NumberUnits,limit,normal_operation,setup_cost,maintenance_penalty,failure_penalty,system_penalty,nN,mean(results),var(results)^0.5])
        
tock() 
    end
    println("-------------------------")
end
end
end
end

60K : 

[ Info:  started timer at: 2024-03-17T20:23:48.827


2
[0 -200 -100 -1200 -2000]
Finding best nN policy...
Max rewards of (N, n) policy is -100315.81104375812  n is 6  N is 6
60
5752.830743633014
-76883.81622237063
60K : 2
[0 -400 -100 -1200 -2000]
Finding best nN policy...


[ Info:         3766.4046763s: 1 hour, 2 minutes, 46 seconds, 404 milliseconds
[ Info:  started timer at: 2024-03-17T21:26:35.531


Max rewards of (N, n) policy is -104075.8175516111  n is 6  N is 6
60
5778.422878882232
-80708.13097445577
60K : 2
[0 -600 -100 -1200 -2000]
Finding best nN policy...


...

[ Info:         3498.6977817s: 58 minutes, 18 seconds, 697 milliseconds
[ Info:  started timer at: 2024-03-20T07:38:55.300


60K : 10
[0 -1800 -100 -1200 -2000]
Finding best nN policy...
Max rewards of (N, n) policy is -117350.31504984514  n is 5  N is 8
60
2931.3829545289855
-98184.79937527623
60K : 10
[0 -2000 -100 -1200 -2000]
Finding best nN policy...


[ Info:          3488.936362s: 58 minutes, 8 seconds, 936 milliseconds
[ Info:  started timer at: 2024-03-20T08:37:04.314


Max rewards of (N, n) policy is -121096.2103372097  n is 5  N is 9
60
3430.068098254444
-100632.3984748372
60K : 10
[0 -2400 -100 -1200 -2000]
Finding best nN policy...


[ Info:         3501.6396532s: 58 minutes, 21 seconds, 639 milliseconds
[ Info:  started timer at: 2024-03-20T09:35:26.021


In [ ]:
df

In [ ]:
using CSV
CSV.write("Hetero (n,N)-policy.csv", df)

In [ ]:
# Transition_matrix = mean(T[:,:,1,1:3],dims=3);
# Transition_matrix=(Transition_matrix./sum(Transition_matrix,dims=2))
# global state_number = state_cnt(NumberUnits,Number_level);

# global crd = Array{Categorical}(undef,Number_level)
# for i in 1:Number_level
#     global crd[i] = Categorical(Transition_matrix[i,:]);
# end

# global multiunit2 = QuickMDP(
#     gen = function (s, a, rng)       #s is a vector of number units at each level and a is the number of units we will repair
#         local_s = s.state;
#         # println(local_s)
#         # based on s, create a status vector corresponding to each unit
#         degradation_state = repeat(1:1,NumberUnits);
#         k = 1;
#         for i in 1:Number_level
#             for j in 1:local_s[i]
#                 degradation_state[k]=i;
#                 k = k+1;
#             end
#         end
#         r = 0.0;
#         prevent_repair = false;
#         if a!=0
#         number_reset = sum(local_s[a:Number_level]);
#         else
#         number_reset = local_s[Number_level];
#         end
#         #using a for loop to compute next state for each unit

#         for i in 1:(NumberUnits-number_reset)  #a is the number of units we want to preventively repair
#             #in this loop, all units continues
#             degradation_state[i] = rand(crd[degradation_state[i]]);
#             r = r+normal_operation;
#         end
        
#         for i in (NumberUnits-number_reset+1):NumberUnits
#             if degradation_state[i] == Number_level
#                 r = r + failure_penalty;
#                 if prevent_repair == false
#                     r = r+setup_cost;
#                     prevent_repair = true;
#                 end
#             else
#                 r = r + maintenance_penalty;
#                 if prevent_repair == false
#                     r = r+setup_cost;
#                     prevent_repair = true;
#                 end
#             end
#             degradation_state[i] = rand(crd[1]); #reset status; add additional transition
#             r = r+normal_operation; #add operation benefit
#          end
#         #collect degradation state to form the state
#         sp = repeat(0:0,Number_level);
#         for i in 1:NumberUnits
#             sp[degradation_state[i]] = sp[degradation_state[i]]+1;
#         end
#         return (sp=mystate(sp), r=r)
#     end,
#     actions = 0:(Number_level-1), 
#     actiontype = function()
#         return Int64;
#     end,

#     initialstate = function()
#         POMDPModelTools.ImplicitDistribution() do rng
#             return (mystate(state_vec(NumberUnits, Number_level, 1)))
#         end
#     end, #all u #all units start fresh. Need to change according to unit number and level number. ##For simulation, we need to use ImplicitDistribution
#     discount = 0.95,
#     isterminal = false              # no ending
# )